In [136]:
!pip install pyspark

In [137]:
from pyspark.sql import SparkSession

In [228]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [230]:
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [231]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [232]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [233]:
from pyspark.sql.functions import length

In [234]:
data = data.withColumn('length',length(data['text']))

In [235]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [236]:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [237]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [238]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [239]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [240]:
from pyspark.ml.classification import NaiveBayes

In [241]:
nb = NaiveBayes()

In [242]:
from pyspark.ml import Pipeline

In [243]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [244]:
cleaner = data_prep_pipe.fit(data)

In [245]:
clean_data = cleaner.transform(data)

In [246]:
clean_data = clean_data.select(['label','features'])

In [247]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [248]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [249]:
spam_predictor = nb.fit(training)

In [250]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [251]:
test_results = spam_predictor.transform(testing)

In [252]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-880.46793865985...|[1.0,2.8704041376...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1173.3950241681...|[1.0,2.4298070857...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-661.89341466972...|[1.0,1.2759625607...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-669.51787603326...|[1.0,1.5505191489...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-844.55430109586...|[1.0,1.7339150724...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-969.34019749141...|[1.0,2.1241852840...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-355.42255796112...|[1.0,1.8482204443...|       0.0|
|  0.0|(13424,[0,1,30,11...|[-592.49524782398...|[1.0,7.2651825971...|       0.0|
|  0.0|(13424,[0,1,500,5...|[-322.00186635670...|[0.99999999997050...|       0.0|
|  0.0|(13424,[0

In [253]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [254]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9278251553773491
